In [8]:
!pip3 install transformers
!pip3 install datasets
!pip3 install huggingsound

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:0

In [9]:
import torch
import librosa
import IPython.display as ipd
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset

In [10]:
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-arabic")
model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-arabic")

test_dataset = load_dataset("common_voice", "ar", split=f"test[:10]")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating train split:   0%|          | 0/14227 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7622 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7517 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/18283 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/43291 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/6333 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/ar/6.1.0/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf. Subsequent calls will reuse this data.


In [11]:
def speech_file_to_array(batch):
    speech_array, sampling_rate = librosa.load(batch["path"], sr=16_000)
    batch["speech"] = speech_array
    batch["sentence"] = batch["sentence"].upper()
    return batch

test_dataset = test_dataset.map(speech_file_to_array)
inputs = processor(test_dataset["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [12]:
with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)

for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Reference:", test_dataset[i]["sentence"])
    print("Prediction:", predicted_sentence)


----------------------------------------------------------------------------------------------------
Reference: ما أطول عودك!
Prediction: ما أطول عودك
----------------------------------------------------------------------------------------------------
Reference: ماتت عمتي منذ سنتين.
Prediction: ماتت عمتي منذ سنتين
----------------------------------------------------------------------------------------------------
Reference: الألمانية ليست لغة سهلة.
Prediction: الألمانية ليست لغة سهلة
----------------------------------------------------------------------------------------------------
Reference: طلبت منه أن يبعث الكتاب إلينا.
Prediction: طلبت منه أن يبعث الكتاب إلينا
----------------------------------------------------------------------------------------------------
Reference: .السيد إيتو رجل متعلم
Prediction: السيد إيتو رجل متعلم
----------------------------------------------------------------------------------------------------
Reference: الحمد لله.
Prediction: الحمد لله
--------------

In [13]:
audio_data_informal = '/content/sample_data/SalmaNewegy_informal-audio_2023-04-16.mp4'
audio_data_formal = '/content/sample_data/SalmaNewegy_formal-audio_2023-04-16.mp4'

In [14]:
ipd.Audio(audio_data_informal)

In [15]:
ipd.Audio(audio_data_formal)

In [16]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-arabic")
audio_paths = [ audio_data_informal, audio_data_formal]

transcriptions = model.transcribe(audio_paths)

print([t["transcription"] for t in transcriptions])

INFO:huggingsound.speech_recognition.model:Loading model...
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
100%|██████████| 2/2 [00:09<00:00,  4.78s/it]

['الخرف الوفكر  بذاخطر على نفسه اللحوليه', 'الخروفة الذي يفكر يصير ختراً على نفسه على الآخرين']


In [18]:
print(transcriptions[1]["transcription"])

الخروفة الذي يفكر يصير ختراً على نفسه على الآخرين


In [19]:
ar_text = transcriptions[1]["transcription"]

In [ ]:
#----------------------------------------------------

In [1]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.8 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


In [4]:
#text = """الخروف الذي يفكر يصير خطراً على نفسه و على الآخرين"""


In [20]:
documentAssembler = DocumentAssembler()\
  .setInputCol("ar_text")\
  .setOutputCol("document")

## More accurate Sentence Detection using Deep Learning
sentencerDL = SentenceDetectorDLModel()\
  .pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentences")

marian = MarianTransformer.pretrained("opus_mt_ar_en", "xx")\
  .setInputCols(["sentences"])\
  .setOutputCol("translation")

nlp_pipeline = Pipeline(
    stages=[
        documentAssembler, 
        sentencerDL, 
        marian
        ])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_ar_en download started this may take some time.
Approximate size to download 390.7 MB
[OK!]


In [21]:
empty_df = spark.createDataFrame([[""]]).toDF('ar_text')
pipeline_model = nlp_pipeline.fit(empty_df)
lmodel = LightPipeline(pipeline_model)
res = lmodel.fullAnnotate(ar_text)


Before _validateStagesInputCols


In [22]:
print ('Original:', ar_text, '\n\n')

print ('Translated:\n')
for sentence in res[0]['translation']:
  print (sentence.result)

Original: الخروفة الذي يفكر يصير ختراً على نفسه على الآخرين 


Translated:

A sheep that thinks it's a traitor on itself over others.
